In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17337,2024-12-04,Brasil Nbb,19:30,Paulistano,R10 Score Vasco da Gama,1.42,2.56,145.5,1.86,1.80,-5.5,1.93,1.74,pMFC4gmR,0.704225,0.390625,0.537634,0.555556,0.094850,0.6,0.4,NaN,NaN,121.088,43.834559,0.362006,136.398,43.400761,0.318192,115.758,135.820,100.74,184.80,0.0,0.0,0.0,0.0,0.405076,0.023184,0.073215,-2.70,-0.540,-0.777778,0.655546,0.3,-0.355546,2.76,0.552,2.826087,0.493475,0.6,0.106525
17338,2024-12-04,Eua Nba,00:00,Denver Nuggets,Golden State Warriors,1.48,2.78,236.5,1.84,2.00,-4.5,1.89,1.97,4IMBnSgn,0.675676,0.359712,0.543478,0.500000,0.035388,1.0,0.4,NaN,NaN,212.888,61.884094,0.290689,187.638,55.279750,0.294608,226.762,253.606,186.44,247.80,0.0,0.0,0.0,0.0,0.431568,0.058926,0.029310,0.95,0.190,2.526316,0.648253,0.5,-0.148253,-1.64,-0.328,-5.426829,0.515336,0.6,0.084664
17339,2024-12-04,Eua Nba,00:00,Sacramento Kings,Houston Rockets,1.81,2.06,226.5,1.81,2.04,-2.5,2.04,1.81,xWKJp6Ob,0.552486,0.485437,0.552486,0.490196,0.037923,0.6,0.6,NaN,NaN,173.376,53.515804,0.308669,187.500,64.938387,0.346338,183.724,266.228,183.75,179.34,0.0,0.0,0.0,0.0,0.091357,0.084485,0.084485,-3.68,-0.736,-1.100543,0.693676,0.3,-0.393676,2.75,0.550,1.927273,0.526944,0.6,0.073056
17340,2024-12-04,Eua Nba,00:30,Los Angeles Clippers,Portland Trail Blazers,1.28,3.93,219.5,1.81,2.04,-9.5,2.02,1.83,YcESrpgB,0.781250,0.254453,0.552486,0.490196,0.035703,0.2,0.2,NaN,NaN,265.536,90.447673,0.340623,532.016,149.913202,0.281783,243.566,614.064,291.06,707.94,0.0,1.0,0.0,0.0,0.719322,0.084485,0.069792,6.26,1.252,0.223642,0.513900,0.7,0.186100,1.54,0.308,9.512987,0.232919,0.3,0.067081
17341,2024-12-04,Europa Euroliga,15:00,Monaco,Olympiakos,1.83,1.99,165.5,1.91,1.91,-1.5,1.95,1.85,tGoouMQq,0.546448,0.502513,0.523560,0.523560,0.048961,1.0,0.8,NaN,NaN,169.878,68.854912,0.405320,143.788,60.363483,0.419809,118.208,157.800,124.63,133.70,0.0,0.0,0.0,0.0,0.059234,0.000000,0.037216,-1.22,-0.244,-3.401639,0.742068,0.7,-0.042068,-0.95,-0.190,-5.210526,0.511781,0.4,-0.111781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17490,2024-12-04,Rússia Copa Da Rússia,13:00,Lokomotiv Kuban 2,Chelyabinsk,1.51,17.50,155.5,1.86,1.88,-6.5,1.99,1.64,AaRnDX74,0.662252,0.057143,0.537634,0.531915,-0.280605,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,113.22,988.00,0.0,1.0,0.0,0.0,1.189546,0.007563,0.136357,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17491,2024-12-04,Rússia Copa Da Rússia,13:00,Tambov,M. Khimki,3.70,18.00,153.5,1.87,1.86,5.5,2.03,1.61,pdGOGzIj,0.270270,0.055556,0.534759,0.537634,-0.674174,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,187.00,1037.00,0.0,1.0,0.0,0.0,0.931947,0.003791,0.163178,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17492,2024-12-04,Rússia Copa Da Rússia,11:00,Samara F,Nika Siktivkar F,1.73,15.00,138.5,1.85,1.85,-3.5,2.02,1.61,2gjE6cjB,0.578035,0.066667,0.540541,0.540541,-0.355299,0.0,0.0,NaN,NaN,543.936,523.708301,0.962812,NaN,NaN,NaN,0.000,0.000,82.14,1488.00,0.0,1.0,0.0,0.0,1.121734,0.000000,0.159732,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17493,2024-12-04,Rússia Copa Da Rússia,12:00,Spartak Moscow F,MBA Moscow F,5.15,19.00,137.5,1.85,1.85,8.5,2.03,1.60,8EgM0SJk,0.194175,0.052632,0.540541,0.540541,-0.753194,0.0,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Denver Nuggets,Golden State Warriors,236.5,1.84,0.9690,Over
1,00:00,Eua Nba,Sacramento Kings,Houston Rockets,226.5,1.81,0.8500,Over
2,00:30,Eua Nba,Los Angeles Clippers,Portland Trail Blazers,219.5,1.81,0.8916,Over
3,09:30,Rússia Liga Vtb United,Enisey,Saratov,150.5,1.86,0.9990,Over
4,21:30,Eua Nba,Boston Celtics,Detroit Pistons,220.5,1.84,0.9868,Over
5,21:30,Eua Nba,Miami Heat,Los Angeles Lakers,216.5,1.84,0.9732,Over
6,21:30,Eua Nba,Philadelphia 76ers,Orlando Magic,207.5,1.81,0.9856,Over
7,22:00,Eua Nba,Milwaukee Bucks,Atlanta Hawks,236.5,1.85,0.9931,Over
